# Testing Code

In [1]:
import cudf

In [3]:
bureau_balance = cudf.read_csv('data/bureau_balance.csv')
bureau = cudf.read_csv('data/bureau.csv')
cc_balance = cudf.read_csv('data/credit_card_balance.csv')
payments = cudf.read_csv('data/POS_CASH_balance.csv')
pc_balance = cudf.read_csv('data/POS_CASH_balance.csv')
prev = cudf.read_csv('data/previous_application.csv')
train = cudf.read_csv('data/application_train.csv')
test = cudf.read_csv('data/application_test.csv')

In [17]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [31]:
bureau_balance.query("SK_ID_BUREAU==5715448")

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
5,5715448,-5,C
6,5715448,-6,C
7,5715448,-7,C
8,5715448,-8,C
9,5715448,-9,0


In [26]:
bureau_balance.groupby('SK_ID_BUREAU').agg({"MONTHS_BALANCE": ['mean', 'max', 'min', 'sum', 'std'], "STATUS": 'max'})

MONTHS_BALANCE                            STATUS
                       mean max min   sum          std    max
SK_ID_BUREAU                                                 
6728373               -12.5   0 -25  -325   7.64852927      X
6743180               -39.0 -33 -45  -507  3.894440482      X
5937081               -37.0   0 -74 -2775  21.79449472      X
5969562               -44.0 -31 -57 -1188  7.937253933      C
6731115                -1.5   0  -3    -6  1.290994449      0
...                     ...  ..  ..   ...          ...    ...
5775417               -43.5   0 -87 -3828  25.54734168      X
5915880               -30.5   0 -61 -1891  18.04161855      X
6304270               -20.5   0 -41  -861  12.26784415      C
5016879               -35.5   0 -71 -2556  20.92844954      X
5931787               -27.0   0 -54 -1485  16.02081979      X

[817395 rows x 6 columns]

In [37]:
df_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
train_numeric = train.select_dtypes(exclude = ['object'])


In [54]:
df_train = train_numeric.drop('TARGET', axis=1)
df_target = train['TARGET']

# Modelling Test

In [41]:
import xgboost as xgb

In [55]:
type(df_target)

cudf.core.series.Series

In [56]:
dtrain = xgb.DMatrix(df_train, label=df_target)

In [59]:
params = {
    'objective': 'binary:logistic',
    'max_depth': 3, 
    'learning_rate': 0.1,
    'subsample': 0.9,
    'colsample_bytree': 0.7,
    'tree_method': 'gpu_hist'
}

In [61]:
clf = xgb.train(params, dtrain, num_boost_round=1000)

[14:58:52] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1614190077837/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
